<a href="https://colab.research.google.com/github/Mogaveera/atmanomalydetection/blob/master/anomaly_2_classes_using_imageclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 1.x

import pandas as pd
import numpy as np
import cv2
import os
import h5py
from tqdm import tqdm
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model, Sequential
from keras.layers import Input, LSTM, Dense, Dropout
from keras.layers import Bidirectional
from keras.utils import to_categorical
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping
from keras.utils.io_utils import HDF5Matrix

SEQ_LEN = 30

train_video_index = []
test_video_index = []

Using TensorFlow backend.


In [0]:
!git clone https://github.com/Mogaveera/anomalydetection3.git

Cloning into 'anomalydetection3'...
remote: Enumerating objects: 149, done.
remote: Total 149 (delta 0), reused 0 (delta 0), pack-reused 149
Receiving objects: 100% (149/149), 629.07 MiB | 13.82 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Checking out files: 100% (141/141), done.


In [0]:
def main():

    base_model = load_model('./drive/My Drive/imageclassifier3_1.h5')

    layer_name = 'flatten_1'
    model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer_name).output)

    # Getting the data
    df = get_data('anomalydetection3/Data/data_file.csv')
    
    # Split the dataset into train and test
    train, test = split_train_test(df)
    
    # Encoding the dataset
    train_video_index = make_dataset(train, model, "train")
    test_video_index = make_dataset(test, model,"test")
    return (train_video_index, test_video_index)

In [0]:
def get_data(path, if_pd=False):
    """Load our data from file."""
    names = ['partition', 'class', 'video_name']
    df = pd.read_csv(path,names=names)
    return df

def split_train_test(df):
    partition =  (df.groupby(['partition']))
    un = df['partition'].unique()
    train = partition.get_group(un[0])
    test = partition.get_group(un[1])
    return (train, test)

def preprocess_image(img):
    img = cv2.resize(img, (299,299))
    return preprocess_input(img)
    
    
def video_to_frame(row, model,label_index, phase, not_created):
    input_f = []
    output_y = []
    index = 0
    cap = cv2.VideoCapture(os.path.join("anomalydetection3/Data","anomaly_dataset",str(row["class"].iloc[0]) ,str(row["video_name"].iloc[0]) + ".mp4")) 
    #print(str(row["class"].iloc[0]))
    #print(str(row["video_name"].iloc[0]))
    frameno = 1
    imgs = []
    length = 0
    seq = 12
    while (cap.isOpened()):
      ret, frame = cap.read()
      if ret:
        if length < seq:
          if frameno % 10 == 0:
            frameno = frameno + 1
            frame = preprocess_image(frame)
            frame = image.img_to_array(frame)
            frame = frame / 255
            # features = model.predict(frame)
            imgs.append(frame)
            length = length + 1
          else:
            frameno = frameno + 1
        else:
          seq = seq + 12
          imgs1 = np.array(imgs)
          features = model.predict(imgs1)
          input_f.append(features)
          output_y.append(label_index)
          del imgs[:]
      else:
        break

    if not_created:
      f = h5py.File(phase+'_4'+'.h5', 'w')
      input_f1 = np.array(input_f)
      output_y1 = np.array(output_y)
      index = input_f1.shape[0]
      if index > 0:
        f.create_dataset(phase, data=input_f1, maxshape=(None, 12, 2048))
        f.create_dataset(phase+"_labels", data=output_y1, maxshape=(None,))
        f.close()
    else:
      hf = h5py.File(phase+'_4'+'.h5', 'a')
      input_f1 = np.array(input_f)
      output_y1 = np.array(output_y)
      index = input_f1.shape[0]
      if index > 0:
        hf[phase].resize((hf[phase].shape[0] + input_f1.shape[0]), axis = 0)
        hf[phase][-input_f1.shape[0]:] = input_f1

        hf[phase+"_labels"].resize((hf[phase+"_labels"].shape[0] + output_y1.shape[0]), axis = 0)
        hf[phase+"_labels"][-output_y1.shape[0]:] = output_y1
        hf.close()

    del input_f[:]
    del output_y[:]
    del imgs[:]
    cap.release()
    return index
    
    


def make_dataset(data, model, phase):
    video_index = [0]
    required_classes = ["Arson", "Burglary", "Fighting", "normal"]
   
    not_created = True
    for i in tqdm(range(data.shape[0])):
    # Check whether the given row , is of a class that is required
        if str(data.iloc[[i]]["class"].iloc[0]) in required_classes:
            index = required_classes.index(str(data.iloc[[i]]["class"].iloc[0]))
            # label_index = np.zeros((4))
            # label_index[index] = 1
            if index < 3:
              label_index = 0
            else:
              label_index = 1
            index = video_to_frame(data.iloc[[i]], model,label_index, phase, not_created)
            real_index = video_index[-1] + index
            video_index.append(real_index)
            if real_index > 0:
              not_created = False

    return video_index

In [0]:
train_video_index, test_video_index = main()

100%|██████████| 24/24 [00:19<00:00,  1.30s/it]


In [0]:
x_train = HDF5Matrix('train_4.h5', 'train')
y_train = HDF5Matrix('train_4.h5', 'train_labels')
x_test = HDF5Matrix('test_4.h5', 'test')
y_test = HDF5Matrix('test_4.h5', 'test_labels')

print(x_train.shape)
print(y_train.shape)
print(y_train[783])
print(x_test.shape)
print(y_test.shape)
print(y_test[93])
print(train_video_index)
print(test_video_index)

(965, 12, 2048)
(965,)
1
(245, 12, 2048)
(245,)
1
[0, 0, 3, 6, 10, 13, 16, 20, 21, 23, 26, 29, 32, 37, 42, 46, 49, 57, 61, 64, 68, 68, 73, 78, 83, 86, 90, 94, 96, 99, 101, 104, 105, 106, 107, 112, 114, 118, 123, 124, 128, 131, 132, 134, 136, 145, 150, 156, 160, 164, 167, 171, 173, 176, 177, 180, 197, 198, 208, 211, 222, 226, 241, 247, 248, 259, 277, 283, 292, 295, 330, 348, 374, 390, 423, 428, 435, 436, 453, 457, 477, 482, 485, 504, 513, 536, 539, 547, 559, 562, 571, 594, 602, 606, 646, 659, 669, 679, 705, 718, 752, 771, 783, 798, 800, 808, 816, 821, 824, 831, 845, 878, 884, 900, 921, 965]
[0, 1, 3, 4, 5, 9, 12, 17, 20, 26, 33, 38, 45, 52, 64, 69, 87, 93, 108, 165, 206, 215, 218, 225, 245]


In [0]:
y_test1 = np.array(y_test)
print(y_test1)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [0]:
def lstm():
    """Build a simple LSTM network. We pass the extracted features from
    our CNN to this model predominantly."""
    input_shape = (12, 2048)
    # Model.
    model = Sequential()
    model.add(Bidirectional(LSTM(2048), input_shape=input_shape))
    model.add(Dropout(0.8))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(2, activation='softmax'))
    #model.add(Dense(10, activation='softmax'))"""
    checkpoint = ModelCheckpoint(filepath='models\\checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
    
    tb_callback = TensorBoard(
    log_dir="logs",
    histogram_freq=2,
    write_graph=True
    )
    
    callback_list = [checkpoint]
    
    optimizer = Adam(lr=1e-5, decay=1e-6)
    metrics = ['accuracy', 'top_k_categorical_accuracy']
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    #return model, callback_list
    #model.compile(optimizer = tf.train.AdamOptimizer(),
    #          loss = 'categorical_crossentropy',
    #        metrics=['accuracy'])
    return model


In [0]:
lstm = lstm()
lstm.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 4096)              67125248  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 69,223,938
Trainable params: 69,223,938
Non-trainable params: 0
___________________________________________________________

In [0]:
lstm.fit(x_train, y_train, batch_size = 16, epochs = 20,verbose = 2, shuffle = 'batch')
lstm.save("Anomaly_Recognition_4classes.h5")

Epoch 1/20
 - 11s - loss: 1.4650 - acc: 0.5036
Epoch 2/20
 - 6s - loss: 1.0950 - acc: 0.5731
Epoch 3/20
 - 6s - loss: 0.8736 - acc: 0.6456
Epoch 4/20
 - 6s - loss: 0.8296 - acc: 0.6560
Epoch 5/20
 - 6s - loss: 0.6965 - acc: 0.6953
Epoch 6/20
 - 6s - loss: 0.6807 - acc: 0.6933
Epoch 7/20
 - 6s - loss: 0.5852 - acc: 0.7285
Epoch 8/20
 - 6s - loss: 0.5307 - acc: 0.7627
Epoch 9/20
 - 6s - loss: 0.5816 - acc: 0.7461
Epoch 10/20
 - 6s - loss: 0.5069 - acc: 0.7751
Epoch 11/20
 - 6s - loss: 0.4019 - acc: 0.8104
Epoch 12/20
 - 6s - loss: 0.4521 - acc: 0.7979
Epoch 13/20
 - 6s - loss: 0.3849 - acc: 0.8435
Epoch 14/20
 - 6s - loss: 0.3604 - acc: 0.8383
Epoch 15/20
 - 6s - loss: 0.3526 - acc: 0.8487
Epoch 16/20
 - 6s - loss: 0.2717 - acc: 0.8881
Epoch 17/20
 - 6s - loss: 0.2590 - acc: 0.8922
Epoch 18/20
 - 6s - loss: 0.2894 - acc: 0.8777
Epoch 19/20
 - 6s - loss: 0.2662 - acc: 0.8870
Epoch 20/20
 - 6s - loss: 0.1887 - acc: 0.9244


In [0]:
test_loss, test_acc = lstm.evaluate(x_test, y_test, batch_size=16)
print("accuracy: ", test_acc)

245/245 [==============================] - 0s 2ms/step
accuracy:  0.8285714285714286
